In [1]:
#EDA
import pandas as pd
import pandas_datareader as pdr
import numpy as np

#Visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Date Time
from datetime import date
from datetime import time
from pandas_profiling import ProfileReport

# SKLEARN
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math

In [3]:
key = '6048c14f45df220c823c80330cdcac459048ce57'
df = pdr.get_data_tiingo('TSLA', api_key = key)

In [4]:
df.to_csv('TSLA.csv')

In [5]:
stocks = pd.read_csv('TSLA.csv')

In [6]:
stocks = stocks.reset_index()

In [7]:
stocks.head()

,index,symbol,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,0,TSLA,2017-01-03 00:00:00+00:00,216.99,220.33,210.96,214.86,5923254,43.398,44.066,42.192,42.972,29616270,0.0,1.0
1,1,TSLA,2017-01-04 00:00:00+00:00,226.99,228.00,214.31,214.75,11213471,45.398,45.600,42.862,42.950,56067355,0.0,1.0
2,2,TSLA,2017-01-05 00:00:00+00:00,226.75,227.48,221.95,226.42,5911695,45.350,45.496,44.390,45.284,29558475,0.0,1.0
3,3,TSLA,2017-01-06 00:00:00+00:00,229.01,230.31,225.45,226.93,5527893,45.802,46.062,45.090,45.386,27639465,0.0,1.0
4,4,TSLA,2017-01-09 00:00:00+00:00,231.28,231.92,228.00,228.97,3979484,46.256,46.384,45.600,45.794,19897420,0.0,1.0


In [8]:
stocks.shape

(1257, 15)

In [9]:
# No null values
stocks.isnull().sum()

index          0
symbol         0
date           0
close          0
high           0
low            0
open           0
volume         0
adjClose       0
adjHigh        0
adjLow         0
adjOpen        0
adjVolume      0
divCash        0
splitFactor    0
dtype: int64

In [10]:
stocks.dtypes

index            int64
symbol          object
date            object
close          float64
high           float64
low            float64
open           float64
volume           int64
adjClose       float64
adjHigh        float64
adjLow         float64
adjOpen        float64
adjVolume        int64
divCash        float64
splitFactor    float64
dtype: object

In [11]:
## Clean up the date column
stocks.date.value_counts()
# from this we can see that the time values for all dates are the same so it can be dropped as it provides no additional info


2017-05-19 00:00:00+00:00    1
2019-07-24 00:00:00+00:00    1
2021-04-30 00:00:00+00:00    1
2020-04-08 00:00:00+00:00    1
2021-01-27 00:00:00+00:00    1
                            ..
2021-09-17 00:00:00+00:00    1
2017-11-16 00:00:00+00:00    1
2021-08-24 00:00:00+00:00    1
2021-08-16 00:00:00+00:00    1
2021-03-05 00:00:00+00:00    1
Name: date, Length: 1257, dtype: int64

In [12]:
#Strip the date column of the time value
stocks['date'] = stocks['date'].apply(str).str.strip('00:00:00+00:00')
#Convert Date column to datetype dtype
stocks['date'] = pd.to_datetime(stocks['date'])
# Create time column using stripped time value
stocks['time'] = stocks['date'].dt.time
#Re-organize dataframe
stocks = stocks.reindex(columns = ['index','symbol','date','time','close', 'high','low','open','volume',
                                  'adjClose', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume', 'divCash', 'splitFactor'])

print(stocks.shape)
stocks.columns

(1257, 16)


Index(['index', 'symbol', 'date', 'time', 'close', 'high', 'low', 'open',
       'volume', 'adjClose', 'adjHigh', 'adjLow', 'adjOpen', 'adjVolume',
       'divCash', 'splitFactor'],
      dtype='object')

In [13]:
# looking at the closing stock prices
closeStocks = stocks['close']
closeStocks

0        216.99
1        226.99
2        226.75
3        229.01
4        231.28
         ...   
1252    1008.87
1253    1067.00
1254    1093.94
1255    1088.47
1256    1086.19
Name: close, Length: 1257, dtype: float64

In [14]:
# Visualize how the closing stock prices look
closeStocks.plot()

<AxesSubplot:>

In [15]:
# LSTM are sensitive to scale so we have to scale the data

In [16]:
scaler = MinMaxScaler(feature_range = (0,1))
closeStocks = scaler.fit_transform(np.array(closeStocks).reshape(-1,1))

In [17]:
closeStocks

array([[0.01845828],
       [0.02331317],
       [0.02319665],
       ...,
       [0.44420763],
       [0.44155201],
       [0.4404451 ]])

In [18]:
closeStocks.shape

(1257, 1)

# Data Preprocessing

#### Train and Test

In [19]:
training_size = int(len(closeStocks)*0.7)
test_size = len(closeStocks) - training_size
train_data,test_data = closeStocks[0:training_size, :], closeStocks[training_size:len(closeStocks),:1]

In [20]:
# Converting our array of values into a dataset matrix

In [21]:
import numpy
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-time_step-1):
		a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
		dataX.append(a)
		dataY.append(dataset[i + time_step, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [22]:
# reshape into x=t, t+1, t+2, t+3, 
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test,y_test = create_dataset(test_data, time_step)

In [23]:
print(X_train.shape), print(y_train.shape)
print(X_test.shape), print(y_test.shape)

(778, 100)
(778,)
(277, 100)
(277,)


(None, None)

In [24]:
# reshaping the models
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [25]:
# Create the Stacked LSTM Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [26]:
model = Sequential()
model.add(LSTM(50, return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 50)           10400     
                                                                 
 lstm_1 (LSTM)               (None, 100, 50)           20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 50,851
Trainable params: 50,851
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.fit(X_train,y_train,validation_data=(X_test,y_test), epochs=100,batch_size=64,verbose=1)

Epoch 1/100
13/13 [==============================] - 7s 230ms/step - loss: 0.0054 - val_loss: 0.0073
Epoch 2/100
13/13 [==============================] - 2s 139ms/step - loss: 0.0017 - val_loss: 0.0054
Epoch 3/100
13/13 [==============================] - 2s 130ms/step - loss: 0.0012 - val_loss: 0.0018
Epoch 4/100
13/13 [==============================] - 2s 131ms/step - loss: 8.7025e-04 - val_loss: 0.0023
Epoch 5/100
13/13 [==============================] - 2s 130ms/step - loss: 8.0255e-04 - val_loss: 0.0015
Epoch 6/100
13/13 [==============================] - 2s 135ms/step - loss: 7.0274e-04 - val_loss: 0.0014
Epoch 7/100
13/13 [==============================] - 2s 129ms/step - loss: 7.0149e-04 - val_loss: 0.0014
Epoch 8/100
13/13 [==============================] - 2s 148ms/step - loss: 5.8593e-04 - val_loss: 0.0016
Epoch 9/100
13/13 [==============================] - 3s 208ms/step - loss: 4.9968e-04 - val_loss: 0.0011
Epoch 10/100
13/13 [==============================] - 2s 186ms/step

Epoch 78/100
13/13 [==============================] - 2s 144ms/step - loss: 1.9313e-04 - val_loss: 0.0015
Epoch 79/100
13/13 [==============================] - 2s 134ms/step - loss: 2.5890e-04 - val_loss: 0.0013
Epoch 80/100
13/13 [==============================] - 2s 136ms/step - loss: 2.6471e-04 - val_loss: 7.9574e-04
Epoch 81/100
13/13 [==============================] - 2s 137ms/step - loss: 2.1244e-04 - val_loss: 9.5484e-04
Epoch 82/100
13/13 [==============================] - 2s 191ms/step - loss: 2.2771e-04 - val_loss: 0.0012
Epoch 83/100
13/13 [==============================] - 2s 163ms/step - loss: 2.1857e-04 - val_loss: 8.7283e-04
Epoch 84/100
13/13 [==============================] - 2s 165ms/step - loss: 1.9321e-04 - val_loss: 7.5858e-04
Epoch 85/100
13/13 [==============================] - 3s 197ms/step - loss: 2.0643e-04 - val_loss: 8.0115e-04
Epoch 86/100
13/13 [==============================] - 2s 193ms/step - loss: 1.9765e-04 - val_loss: 0.0011
Epoch 87/100
13/13 [======

In [29]:
import tensorflow as tf

In [30]:
# predicting using the train_data
trainpred = model.predict(X_train)
testpred = model.predict(X_test)

In [31]:
# Transform bark
trainpred = scaler.inverse_transform(trainpred)
testpred = scaler.inverse_transform(testpred)

In [32]:
# Calculate metrics 
# For Train
trainMSE = mean_squared_error(y_train, trainpred)
trainRMSE = math.sqrt(trainMSE)

# For Test 
testMSE = mean_squared_error(y_test, testpred)
testRMSE = math.sqrt(testMSE)

print(trainRMSE, testRMSE)

399.4202449750937 783.4419035381625


In [33]:
### Plotting 
# shift train predictions for plotting
look_back=100
trainPredictPlot = numpy.empty_like(closeStocks)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainpred)+look_back, :] = trainpred

# shift test predictions for plotting
testPredictPlot = numpy.empty_like(closeStocks)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainpred)+(look_back*2)+1:len(closeStocks)-1, :] = testpred

# plot baseline and predictions
plt.plot(scaler.inverse_transform(closeStocks))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.show()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning:

Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.



In [34]:
len(test_data)

378

In [ ]:
x_input=test_data[341:].reshape(1,-1)

In [ ]:
temp_input= list(x_input)
temp_input = temp_input[0].tolist()

In [ ]:
# demonstrate prediction for next 10 days
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        #print(temp_input)
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        preds = model.predict(x_input, verbose=0)
        print(yhat[0])
        temp_input.extend(predst[0].tolist())
        print(len(temp_input))
        lst_output.extend(preds.tolist())
        i=i+1
    
    

print(lst_output)

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
len(closeStocks)

In [ ]:
plt.plot(day_new,scaler.inverse_transform(closeStocks[1157:]))
plt.plot(day_pred,scaler.inverse_transform(lst_output))

In [ ]:
closeplot=closeStocks.tolist()
closeplot.extend(lst_output)
plt.plot(closeplot[1200:])

In [ ]:
closeplot=scaler.inverse_transform(closeplot).tolist()

In [ ]:
plt.plot(closeplot)